Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 03. Train on Azure Container Instance (EXPERIMENTAL)

* Create Workspace
* Create Project
* Create `train.py` in the project folder.
* Configure an ACI (Azure Container Instance) run
* Execute in ACI

## Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'train-on-aci'
experiment = Experiment(workspace = ws, name = experiment_name)

Create a folder to store the training script.

In [ ]:
import os
script_folder = './samples/train-on-aci'
os.makedirs(script_folder, exist_ok = True)

## Remote execution on ACI

Use `%%writefile` magic to write training code to `train.py` file under the project folder.

In [ ]:
%%writefile $script_folder/train.py

import os
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from sklearn.externals import joblib

import numpy as np

os.makedirs('./outputs', exist_ok=True)

X, y = load_diabetes(return_X_y = True)

run = Run.get_submitted_run()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
data = {"train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

# list of numbers from 0.0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

for alpha in alphas:
    # Use Ridge algorithm to create a regression model
    reg = Ridge(alpha = alpha)
    reg.fit(data["train"]["X"], data["train"]["y"])

    preds = reg.predict(data["test"]["X"])
    mse = mean_squared_error(preds, data["test"]["y"])
    run.log('alpha', alpha)
    run.log('mse', mse)
    
    model_file_name = 'ridge_{0:.2f}.pkl'.format(alpha)
    with open(model_file_name, "wb") as file:
        joblib.dump(value = reg, filename = 'outputs/' + model_file_name)

    print('alpha is {0:.2f}, and mse is {1:0.2f}'.format(alpha, mse))

## Configure for using ACI
Linux-based ACI is available in `westus`, `eastus`, `westeurope`, `northeurope`, `westus2` and `southeastasia` regions. See details [here](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-quotas#region-availability).

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration()

# signal that you want to use ACI to execute script.
run_config.target = "containerinstance"

# ACI container group is only supported in certain regions, which can be different than the region the Workspace is in.
run_config.container_instance.region = 'eastus'

# set the ACI CPU and Memory 
run_config.container_instance.cpu_cores = 1
run_config.container_instance.memory_gb = 2

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
#run_config.environment.docker.base_image = 'microsoft/mmlspark:plus-0.9.9'

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

## Submit the Experiment
Finally, run the training job on the ACI

In [ ]:
%%time 
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory = script_folder,
                                    script= 'train.py',
                                    run_config = run_config)

run = experiment.submit(script_run_config)


In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output = True)

In [ ]:
# Show run details
run

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

In [ ]:
import numpy as np
print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    min(metrics['mse']), 
    metrics['alpha'][np.argmin(metrics['mse'])]
))